# Gradient Boosted Trees

In [30]:
from fastai.tabular.all import *
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier

path = Path('/Users/baranserajelahi/Codes/fraud-detection-pytorch-scikit-fastai/data')
Path.BASE_PATH = path
pd.options.display.max_rows = 200
pd.options.display.max_columns = 120

In [31]:
# Tabular object created and saved on the 02_ notebook
to = (path/'to.pkl').load()

In [32]:
cats = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 
       'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 
       'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType', 'DeviceInfo', 'id_12', 
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 
       'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38']

for cat in cats:
    to[cat] = to[cat].astype('category')

In [33]:
xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

## HistGradientBoostingClassifier

In [112]:
'Previously, leaf-wise growth was an exclusive feature of lightGBM, but xgboost has since implemented this growth strategy (this change has not been reflected in the lightGBM docs, but has been acknowledged in a blog post). This strategy is only available for the histogram-based method (which I will explain below), so in order to use it, users will have to set the tree_method parameter to hist and the grow_policy parameter to lossguide.'

'Previously, leaf-wise growth was an exclusive feature of lightGBM, but xgboost has since implemented this growth strategy (this change has not been reflected in the lightGBM docs, but has been acknowledged in a blog post). This strategy is only available for the histogram-based method (which I will explain below), so in order to use it, users will have to set the tree_method parameter to hist and the grow_policy parameter to lossguide.'

In [113]:
'Xgboost offers the option tree_method=approx, which computes a new set of bins at each split using the gradient statistics. LightGBM and xgboost with the tree_method set to hist will both compute the bins at the beginning of training and reuse the same bins throughout the entire training process.'

'Xgboost offers the option tree_method=approx, which computes a new set of bins at each split using the gradient statistics. LightGBM and xgboost with the tree_method set to hist will both compute the bins at the beginning of training and reuse the same bins throughout the entire training process.'

In [114]:
'BOTH: Though lightGBM does not enable ignoring zero values by default, it has an option called zero_as_missing which, if set to True, will regard all zero values as missing. According to this thread on GitHub, lightGBM will treat missing values in the same way as xgboost as long as the parameter use_missing is set to True (which is the default behavior).'

'Though lightGBM does not enable ignoring zero values by default, it has an option called zero_as_missing which, if set to True, will regard all zero values as missing. According to this thread on GitHub, lightGBM will treat missing values in the same way as xgboost as long as the parameter use_missing is set to True (which is the default behavior).'

In [115]:
'LGBM: This is a method introduced in lightGBM that also takes advantage of the sparsity of large datasets. The essential observation behind this method is that the sparsity of features means that some features are never non-zero together. For instance, the words "Python" and "protein" might never appear in the same document in the data. This means that these features can be "bundled" into a single feature without losing any information. Suppose the tf-idf score for "Python" ranges from 0 to 10 and the tf-idf score for  "protein" ranges from 0 to 20. In this case, the feature'

'This is a method introduced in lightGBM that also takes advantage of the sparsity of large datasets. The essential observation behind this method is that the sparsity of features means that some features are never non-zero together. For instance, the words "Python" and "protein" might never appear in the same document in the data. This means that these features can be "bundled" into a single feature without losing any information. Suppose the tf-idf score for "Python" ranges from 0 to 10 and the tf-idf score for  "protein" ranges from 0 to 20. In this case, the feature'

In [116]:
'LGBM: Unfortunately, the problem of finding the most efficient bundle is NP-hard. Therefore, the authors of the paper opted for an approximate algorithm that tolerated a certain degree of overlap \gamma  between the non-zero elements within a feature bundle. The details of this algorithm are beyond the scope of this post, so please refer to the original paper for details.'

'Unfortunately, the problem of finding the most efficient bundle is NP-hard. Therefore, the authors of the paper opted for an approximate algorithm that tolerated a certain degree of overlap \\gamma  between the non-zero elements within a feature bundle. The details of this algorithm are beyond the scope of this post, so please refer to the original paper for details.'

HistGradientBoostingClassifier is based on Microsofts LightGBM.

### Default parameters 

In [34]:
def m_roc(y, xs):
    preds = clf.predict_proba(xs)
    preds_isFraud = preds[:,1]
    return roc_auc_score(y, preds_isFraud)

The max_iter parameter was increased to 1000 from its default value of 100.

In [35]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1,
                                    max_iter=1000, scoring='roc_auc')
clf.fit(xs,y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.272 s
Binning 0.005 GB of validation data: 0.018 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.81704, val score: 0.80557, in 0.046s
[2/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.83950, val score: 0.81193, in 0.049s
[3/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.85244, val score: 0.81582, in 0.045s
[4/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.85856, val score: 0.81738, in 0.049s
[5/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.86801, val score: 0.83294, in 0.050s
[6/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.87325, val score: 0.83324, in 0.053s
[7/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.87697, val score: 0.83377, in 0.066s
[8/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.88005, val score: 0.83974, in 0.049s
[9/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.88087, val score: 0.84503, in 0.054s
[10/1000] 1 tree, 31 l

[85/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.98212, val score: 0.88056, in 0.079s
[86/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.98236, val score: 0.88083, in 0.094s
[87/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.98245, val score: 0.88074, in 0.083s
[88/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.98260, val score: 0.88102, in 0.073s
[89/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.98272, val score: 0.88450, in 0.068s
[90/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.98285, val score: 0.88506, in 0.093s
[91/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.98320, val score: 0.88526, in 0.126s
[92/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.98328, val score: 0.88487, in 0.069s
[93/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.98338, val score: 0.88525, in 0.090s
[94/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.98419, val score: 0.88511, in 0.086s
[95/1000] 1 tree, 31 leaves, m

(0.981080736425216, 0.8585535814408465)

The algorithm may have stopped too early, so let's increase the threshold for early stopping by decreasing tol by a factor of 10 to e-8 from its default value e-7.

In [36]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1,
                                     max_iter=1000, scoring='roc_auc', tol=1e-8)
clf.fit(xs,y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.271 s
Binning 0.005 GB of validation data: 0.019 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 7, train score: 0.82234, val score: 0.80514, in 0.047s
[2/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.84012, val score: 0.83320, in 0.056s
[3/1000] 1 tree, 31 leaves, max depth = 11, train score: 0.84453, val score: 0.83291, in 0.047s
[4/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.85679, val score: 0.84107, in 0.049s
[5/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.86389, val score: 0.84776, in 0.052s
[6/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.86967, val score: 0.85248, in 0.052s
[7/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.87200, val score: 0.85335, in 0.059s
[8/1000] 1 tree, 31 leaves, max depth = 9, train score: 0.87481, val score: 0.85439, in 0.058s
[9/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.87650, val score: 0.85888, in 0.058s
[10/1000] 1 tree, 31

[85/1000] 1 tree, 31 leaves, max depth = 14, train score: 0.97995, val score: 0.92332, in 0.078s
[86/1000] 1 tree, 31 leaves, max depth = 14, train score: 0.98003, val score: 0.92413, in 0.082s
[87/1000] 1 tree, 31 leaves, max depth = 15, train score: 0.98041, val score: 0.92391, in 0.076s
[88/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.98046, val score: 0.92400, in 0.080s
[89/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.98087, val score: 0.92384, in 0.073s
[90/1000] 1 tree, 31 leaves, max depth = 14, train score: 0.98111, val score: 0.92433, in 0.079s
[91/1000] 1 tree, 31 leaves, max depth = 8, train score: 0.98120, val score: 0.92435, in 0.076s
[92/1000] 1 tree, 31 leaves, max depth = 10, train score: 0.98128, val score: 0.92427, in 0.078s
[93/1000] 1 tree, 31 leaves, max depth = 12, train score: 0.98138, val score: 0.92451, in 0.076s
[94/1000] 1 tree, 31 leaves, max depth = 14, train score: 0.98177, val score: 0.92490, in 0.080s
[95/1000] 1 tree, 31 leaves, ma

(0.9852662994607253, 0.8569054436362248)

### Hyperparameter tuning

Hyperparameters were tuned interatively. Using both GridSearchCV and RandomizedGridSearchCV. RandomizedGridSearchCV was used for all 5 sets of parameters, than the final parameter set was double checked using GridSearchCV. Only the final results are displayed in the notenbook.

In [30]:
parameters1 = {
 'max_iter': [1000],
 'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
 'max_depth': [None, 25, 50, 75],
 'l2_regularization': [0, 1.5],
 'max_bins': [255],
 'max_leaf_nodes': [20, 31, 40, 50, 60],
 'min_samples_leaf': [1, 5, 10, 20],
 'random_state': [33],
 }

In [7]:
parameters2 = {
 'max_iter': [1000],
 'learning_rate': [0.01, 0.1, 0.2, 0.3],
 'max_depth': [None, 25, 50],
 'l2_regularization': [0, 1.5],
 'max_bins': [255],
 'max_leaf_nodes': [20, 31, 40],
 'min_samples_leaf': [1, 5],
 'random_state': [33],
 }

In [23]:
parameters3 = {
 'max_iter': [1000],
 'learning_rate': [0.005, 0.01, 0.1],
 'max_depth': [15, 25, 35],
 'l2_regularization': [1.5, 2.0],
 'max_bins': [255],
 'max_leaf_nodes': [40, 45, 50],
 'min_samples_leaf': [5, 7, 10],
 'random_state': [33],
 }

In [31]:
parameters4 = {
 'max_iter': [1000],
 'learning_rate': [0.01, 0.02],
 'max_depth': [25, 30],
 'l2_regularization': [2.0, 2.5, 3.0],
 'max_bins': [255],
 'max_leaf_nodes': [37, 40, 43],
 'min_samples_leaf': [7, 8, 9],
 'random_state': [33],
 }

In [36]:
parameters5 = {
 'max_iter': [1000],
 'learning_rate': [0.01, 0.02, 0.03],
 'max_depth': [25, 27],
 'l2_regularization': [2.4, 2.5, 2.6],
 'max_bins': [255],
 'max_leaf_nodes': [43, 44, 45],
 'random_state': [33],
 }

#### GridSearchCV

In [37]:
clf = GridSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000, 
                                                  scoring='roc_auc', tol=1e-8 ), parameters5, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [38]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 1min 24s, sys: 22.1 s, total: 1min 46s
Wall time: 11min 42s


(HistGradientBoostingClassifier(l2_regularization=2.4, learning_rate=0.03,
                                loss='binary_crossentropy', max_depth=25,
                                max_iter=1000, max_leaf_nodes=44,
                                random_state=33, scoring='roc_auc', tol=1e-08),
 0.8691008607988312,
 {'l2_regularization': 2.4,
  'learning_rate': 0.03,
  'max_bins': 255,
  'max_depth': 25,
  'max_iter': 1000,
  'max_leaf_nodes': 44,
  'random_state': 33})

In [42]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=25, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.273 s
Binning 0.005 GB of validation data: 0.019 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.83422, val score: 0.83526, in 0.076s
[2/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.84166, val score: 0.84216, in 0.081s
[3/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.84365, val score: 0.84422, in 0.081s
[4/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.84635, val score: 0.84409, in 0.120s
[5/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.85918, val score: 0.85764, in 0.079s
[6/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.85980, val score: 0.85770, in 0.122s
[7/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.85984, val score: 0.85860, in 0.120s
[8/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.86009, val score: 0.85848, in 0.130s
[9/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.86031, val score: 0.85919, in 0.116s
[10/1000] 1 tree,

[85/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.92742, val score: 0.90149, in 0.116s
[86/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.92801, val score: 0.90142, in 0.117s
[87/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.92856, val score: 0.90216, in 0.111s
[88/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.92926, val score: 0.90224, in 0.111s
[89/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.92989, val score: 0.90258, in 0.107s
[90/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.93051, val score: 0.90300, in 0.181s
[91/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.93145, val score: 0.90335, in 0.106s
[92/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.93208, val score: 0.90435, in 0.272s
[93/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.93265, val score: 0.90504, in 0.110s
[94/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.93347, val score: 0.90482, in 0.107s
[95/1000] 1 tree, 44 leaves, m

[169/1000] 1 tree, 44 leaves, max depth = 20, train score: 0.96254, val score: 0.91969, in 0.204s
[170/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.96271, val score: 0.91994, in 0.156s
[171/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96299, val score: 0.91994, in 0.145s
[172/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96309, val score: 0.92024, in 0.142s
[173/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96335, val score: 0.92045, in 0.171s
[174/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96377, val score: 0.92087, in 0.175s
[175/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.96408, val score: 0.92078, in 0.150s
[176/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96426, val score: 0.92085, in 0.141s
[177/1000] 1 tree, 44 leaves, max depth = 21, train score: 0.96446, val score: 0.92099, in 0.145s
[178/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.96457, val score: 0.92096, in 0.211s
[179/1000] 1 tree, 4

(0.9659117013340929, 0.8577956250355605)

#### RandomizedSearchCV

In [43]:
clf = RandomizedSearchCV(HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000, 
                                                  scoring='roc_auc', tol=1e-8 ), parameters5, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [44]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 1min 19s, sys: 18 s, total: 1min 37s
Wall time: 2min 17s


(HistGradientBoostingClassifier(l2_regularization=2.4, learning_rate=0.03,
                                loss='binary_crossentropy', max_depth=27,
                                max_iter=1000, max_leaf_nodes=44,
                                random_state=33, scoring='roc_auc', tol=1e-08),
 0.8691008607988312,
 {'random_state': 33,
  'max_leaf_nodes': 44,
  'max_iter': 1000,
  'max_depth': 27,
  'max_bins': 255,
  'learning_rate': 0.03,
  'l2_regularization': 2.4})

In [45]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=27, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.259 s
Binning 0.005 GB of validation data: 0.019 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.82760, val score: 0.80686, in 0.052s
[2/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.83003, val score: 0.80806, in 0.101s
[3/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.83584, val score: 0.81004, in 0.050s
[4/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.84916, val score: 0.83466, in 0.052s
[5/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.85368, val score: 0.83562, in 0.053s
[6/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.85301, val score: 0.83424, in 0.053s
[7/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.85311, val score: 0.83412, in 0.051s
[8/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.85305, val score: 0.83381, in 0.055s
[9/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.85380, val score: 0.83552, in 0.052s
[10/1000] 1 tre

[85/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.92146, val score: 0.87816, in 0.105s
[86/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.92232, val score: 0.87912, in 0.083s
[87/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.92255, val score: 0.87928, in 0.080s
[88/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.92288, val score: 0.87950, in 0.074s
[89/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.92380, val score: 0.88020, in 0.117s
[90/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.92415, val score: 0.88021, in 0.065s
[91/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.92469, val score: 0.88100, in 0.067s
[92/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.92538, val score: 0.88118, in 0.077s
[93/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.92579, val score: 0.88122, in 0.076s
[94/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.92619, val score: 0.88132, in 0.080s
[95/1000] 1 tree, 44 leaves, m

[252/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.97205, val score: 0.90193, in 0.132s
[253/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.97217, val score: 0.90204, in 0.124s
[254/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.97223, val score: 0.90219, in 0.130s
[255/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.97230, val score: 0.90210, in 0.132s
[256/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.97251, val score: 0.90237, in 0.113s
[257/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.97260, val score: 0.90257, in 0.129s
[258/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.97271, val score: 0.90271, in 0.133s
[259/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.97284, val score: 0.90277, in 0.147s
[260/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.97300, val score: 0.90276, in 0.142s
[261/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.97324, val score: 0.90265, in 0.133s
[262/1000] 1 tree, 4

[336/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.98139, val score: 0.90867, in 0.190s
[337/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.98145, val score: 0.90875, in 0.158s
[338/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.98155, val score: 0.90877, in 0.295s
[339/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.98161, val score: 0.90901, in 0.219s
[340/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.98163, val score: 0.90904, in 0.152s
[341/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.98172, val score: 0.90914, in 0.205s
[342/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.98180, val score: 0.90921, in 0.250s
[343/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.98187, val score: 0.90927, in 0.197s
[344/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.98190, val score: 0.90934, in 0.150s
[345/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.98194, val score: 0.90940, in 0.151s
[346/1000] 1 tree, 4

(0.9795618885673211, 0.865225601216363)

#### Including the V*** columns

In [37]:
to = (path/'to2.pkl').load()

xs, y             = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

In [38]:
cats = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 
       'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 
       'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType', 'DeviceInfo', 'id_12', 
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 
       'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38']

for cat in cats:
    to[cat] = to[cat].astype('category')

In [39]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=27, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.279 GB of training data: 1.763 s
Binning 0.031 GB of validation data: 0.048 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.86433, val score: 0.82418, in 0.213s
[2/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.87088, val score: 0.83679, in 0.230s
[3/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.87225, val score: 0.84385, in 0.256s
[4/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.87238, val score: 0.84300, in 0.539s
[5/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.87293, val score: 0.84293, in 0.397s
[6/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.87305, val score: 0.84355, in 0.408s
[7/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.87715, val score: 0.84655, in 0.485s
[8/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.87793, val score: 0.84680, in 0.225s
[9/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.87836, val score: 0.84739, in 0.232s
[10/1000] 1 tre

[85/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.93783, val score: 0.89626, in 0.380s
[86/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.93845, val score: 0.89685, in 0.365s
[87/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.93886, val score: 0.89716, in 0.350s
[88/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.93966, val score: 0.89760, in 0.355s
[89/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.94004, val score: 0.89739, in 0.357s
[90/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.94061, val score: 0.89757, in 0.602s
[91/1000] 1 tree, 44 leaves, max depth = 21, train score: 0.94094, val score: 0.89835, in 0.666s
[92/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.94140, val score: 0.89865, in 0.642s
[93/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.94170, val score: 0.89937, in 0.620s
[94/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.94215, val score: 0.89951, in 0.940s
[95/1000] 1 tree, 44 leaves, m

[169/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96779, val score: 0.91343, in 0.391s
[170/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.96792, val score: 0.91360, in 0.561s
[171/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.96812, val score: 0.91349, in 0.456s
[172/1000] 1 tree, 44 leaves, max depth = 20, train score: 0.96830, val score: 0.91354, in 0.478s
[173/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96863, val score: 0.91417, in 0.459s
[174/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.96885, val score: 0.91433, in 0.444s
[175/1000] 1 tree, 44 leaves, max depth = 21, train score: 0.96893, val score: 0.91473, in 0.427s
[176/1000] 1 tree, 44 leaves, max depth = 21, train score: 0.96929, val score: 0.91468, in 0.677s
[177/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96939, val score: 0.91492, in 0.737s
[178/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.96957, val score: 0.91510, in 0.735s
[179/1000] 1 tree, 44

[253/1000] 1 tree, 44 leaves, max depth = 20, train score: 0.97987, val score: 0.92167, in 0.760s
[254/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.98000, val score: 0.92181, in 0.696s
[255/1000] 1 tree, 44 leaves, max depth = 22, train score: 0.98009, val score: 0.92190, in 0.775s
[256/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.98023, val score: 0.92193, in 1.145s
[257/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.98042, val score: 0.92209, in 0.430s
[258/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.98048, val score: 0.92213, in 0.440s
[259/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.98054, val score: 0.92219, in 0.443s
[260/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.98062, val score: 0.92238, in 0.464s
[261/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.98076, val score: 0.92245, in 0.462s
[262/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.98087, val score: 0.92256, in 0.466s
[263/1000] 1 tree, 4

[337/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.98687, val score: 0.92591, in 0.725s
[338/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.98700, val score: 0.92603, in 0.690s
[339/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.98706, val score: 0.92607, in 1.038s
[340/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.98709, val score: 0.92620, in 0.805s
[341/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.98721, val score: 0.92622, in 0.812s
[342/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.98733, val score: 0.92639, in 1.163s
[343/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.98736, val score: 0.92632, in 1.388s
[344/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.98738, val score: 0.92626, in 1.273s
[345/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.98741, val score: 0.92622, in 0.814s
[346/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.98744, val score: 0.92621, in 0.605s
[347/1000] 1 tree, 4

[421/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.99123, val score: 0.92969, in 0.502s
[422/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.99136, val score: 0.92972, in 0.547s
[423/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.99140, val score: 0.92957, in 0.563s
[424/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.99142, val score: 0.92949, in 0.584s
[425/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.99145, val score: 0.92944, in 0.586s
[426/1000] 1 tree, 44 leaves, max depth = 17, train score: 0.99145, val score: 0.92948, in 0.525s
[427/1000] 1 tree, 44 leaves, max depth = 19, train score: 0.99150, val score: 0.92943, in 0.530s
[428/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.99155, val score: 0.92950, in 0.749s
[429/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.99158, val score: 0.92944, in 0.829s
[430/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.99160, val score: 0.92945, in 0.824s
[431/1000] 1 tree, 4

(0.9844357994538068, 0.8698689690377148)

## GradientBoostingClassifier 

HistGradientBoostingClassifier is recommended for datasets with a large number of classes. Since this is a binary classification problem, I decided to try the more established GradientBoostingClassifier class also.

In [49]:
from sklearn.ensemble import GradientBoostingClassifier 

In [117]:
GradientBoostingClassifier?

In [58]:
# Tabular object created and saved on the 02_ notebook
to = (path/'to.pkl').load()

xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

### Defaults

n_estimators was increased to 1000 from its default value of 100.

In [61]:
clf = GradientBoostingClassifier(loss='deviance', n_estimators=1000, verbose=1)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

      Iter       Train Loss   Remaining Time 
         1           0.2858            3.30m
         2           0.2787            3.31m
         3           0.2732            3.41m
         4           0.2681            3.42m
         5           0.2632            3.47m
         6           0.2587            3.47m
         7           0.2550            3.47m
         8           0.2521            3.47m
         9           0.2490            3.47m
        10           0.2456            3.46m
        20           0.2276            3.40m
        30           0.2183            3.34m
        40           0.2126            3.27m
        50           0.2085            3.23m
        60           0.2047            3.22m
        70           0.2017            3.20m
        80           0.1991            3.17m
        90           0.1969            3.15m
       100           0.1948            3.12m
       200           0.1771            2.83m
       300           0.1646            2.52m
       40

(0.9705755015972327, 0.8568501495478038)

### Hyperparameter Tuning

In [63]:
parameters1 = {
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'n_estimators': [40, 50, 60, 70, 1000]
}

In [72]:
parameters2 = {
 'max_depth': [3, 6, 9, 12],
 'subsample': [0.8, 0.66],
 'min_samples_split': [2, 5, 10],
 'max_leaf_nodes': [None, 20, 31, 40],
 'min_samples_leaf': [1, 5],
 'random_state': [33],
 }

In [78]:
parameters3 = {
 'max_depth': [8, 9, 10],
 'subsample': [0.60, 0.66, 0.72],
 'min_samples_split': [3, 5, 8],
 'max_leaf_nodes': [None, 45, 50, 60],
 'min_samples_leaf': [5, 7, 9],
 'random_state': [33],
}

In [81]:
parameters4 = {
 'max_depth': [8, 9, 10],
 'subsample': [0.68, 0.72, 0.76],
 'min_samples_split': [1, 2, 3],
 'max_leaf_nodes': [50, 52, 54],
 'min_samples_leaf': [6, 7, 8],
 'random_state': [33],
 }

In [99]:
parameters5 = {
    'n_estimators': [40, 50, 60, 70, 100, 1000]
}

In [100]:
clf = GridSearchCV(GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                         criterion='friedman_mse', tol=1e-8, max_features='sqrt',
                                subsample=0.72, min_samples_split=6, 
                                max_leaf_nodes=52, max_depth=10), 
                         parameters5, 
                   scoring=make_scorer(roc_auc_score, needs_proba=True), n_jobs=-1)

In [101]:
%%time
clf.fit(xs, y)
clf.best_estimator_, clf.best_score_, clf.best_params_

CPU times: user 1min 12s, sys: 1.32 s, total: 1min 13s
Wall time: 4min 14s


(GradientBoostingClassifier(max_depth=10, max_features='sqrt', max_leaf_nodes=52,
                            min_samples_split=6, n_estimators=1000,
                            subsample=0.72, tol=1e-08),
 0.8841218032816766,
 {'n_estimators': 1000})

In [104]:
clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, verbose=1,
                         criterion='friedman_mse', tol=1e-8, max_features='sqrt',
                                subsample=0.72, min_samples_split=6, 
                                max_leaf_nodes=52, max_depth=10, n_estimators=1000)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.2693           0.0407            1.12m
         2           0.2610           0.0080            1.04m
         3           0.2520           0.0075           59.71s
         4           0.2429           0.0055            1.08m
         5           0.2422           0.0028            1.06m
         6           0.2346           0.0033            1.05m
         7           0.2353           0.0050            1.04m
         8           0.2226           0.0042            1.03m
         9           0.2224           0.0027            1.03m
        10           0.2239           0.0020            1.04m
        20           0.1882           0.0006            1.07m
        30           0.1758           0.0002            1.03m
        40           0.1648          -0.0000            1.03m
        50           0.1593          -0.0001            1.03m
        60           0.1503          -0.0010            1.02m
       

(1.0, 0.8318749851413832)

In [103]:
clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.005, verbose=1,
                         criterion='friedman_mse', tol=1e-8, max_features='sqrt',
                                subsample=0.72, min_samples_split=6, 
                                max_leaf_nodes=52, max_depth=10, n_estimators=1000)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.3119           0.0022            1.08m
         2           0.3162           0.0018            1.10m
         3           0.3054           0.0018            1.03m
         4           0.3067           0.0014            1.06m
         5           0.3047           0.0014            1.05m
         6           0.3057           0.0015            1.05m
         7           0.2964           0.0015            1.06m
         8           0.3050           0.0014            1.05m
         9           0.2936           0.0013            1.06m
        10           0.2941           0.0012            1.02m
        20           0.2828           0.0009           58.00s
        30           0.2784           0.0006           58.51s
        40           0.2696           0.0005           58.59s
        50           0.2593           0.0004           57.99s
        60           0.2554           0.0004           58.28s
       

(0.9152136625826836, 0.843631924082018)

In [105]:
clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.01, verbose=1,
                         criterion='friedman_mse', tol=1e-8, max_features='sqrt',
                                subsample=0.72, min_samples_split=6, 
                                max_leaf_nodes=52, max_depth=10, n_estimators=1000)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.3082           0.0047            1.15m
         2           0.3072           0.0027            1.30m
         3           0.3074           0.0027            1.21m
         4           0.2995           0.0030            1.21m
         5           0.3000           0.0023            1.16m
         6           0.2976           0.0022            1.14m
         7           0.2994           0.0018            1.18m
         8           0.2937           0.0017            1.17m
         9           0.2912           0.0017            1.17m
        10           0.2880           0.0020            1.19m
        20           0.2695           0.0011            1.27m
        30           0.2599           0.0008            1.23m
        40           0.2517           0.0007            1.22m
        50           0.2427           0.0005            1.20m
        60           0.2351           0.0004            1.17m
       

(0.9472911054564462, 0.8462935197198218)

In [19]:
to = (path/'to.pkl').load()
xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

In [20]:
cats = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 
       'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 
       'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType', 'DeviceInfo', 'id_12', 
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 
       'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38']

In [21]:
for cat in cats:
    to[cat] = to[cat].astype('category')

In [26]:
HistGradientBoostingClassifier?

In [28]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=25, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)

Binning 0.042 GB of training data: 0.269 s
Binning 0.005 GB of validation data: 0.019 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.84781, val score: 0.81056, in 0.059s
[2/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.85082, val score: 0.81300, in 0.064s
[3/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.85164, val score: 0.81319, in 0.064s
[4/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.87033, val score: 0.83992, in 0.066s
[5/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.87400, val score: 0.84196, in 0.069s
[6/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.87655, val score: 0.84878, in 0.133s
[7/1000] 1 tree, 44 leaves, max depth = 11, train score: 0.87728, val score: 0.84776, in 0.069s
[8/1000] 1 tree, 44 leaves, max depth = 9, train score: 0.87761, val score: 0.84773, in 0.075s
[9/1000] 1 tree, 44 leaves, max depth = 10, train score: 0.87773, val score: 0.84804, in 0.070s
[10/1000] 1 tree

[85/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.93370, val score: 0.87446, in 0.114s
[86/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.93435, val score: 0.87498, in 0.108s
[87/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.93477, val score: 0.87535, in 0.099s
[88/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.93512, val score: 0.87579, in 0.102s
[89/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.93544, val score: 0.87655, in 0.104s
[90/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.93562, val score: 0.87675, in 0.103s
[91/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.93631, val score: 0.87708, in 0.105s
[92/1000] 1 tree, 44 leaves, max depth = 13, train score: 0.93680, val score: 0.87698, in 0.182s
[93/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.93715, val score: 0.87712, in 0.101s
[94/1000] 1 tree, 44 leaves, max depth = 12, train score: 0.93752, val score: 0.87718, in 0.103s
[95/1000] 1 tree, 44 leaves, m

[169/1000] 1 tree, 44 leaves, max depth = 16, train score: 0.96245, val score: 0.88927, in 0.134s
[170/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96257, val score: 0.88926, in 0.136s
[171/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96294, val score: 0.88962, in 0.128s
[172/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96325, val score: 0.88969, in 0.133s
[173/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96354, val score: 0.88955, in 0.132s
[174/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96387, val score: 0.88965, in 0.134s
[175/1000] 1 tree, 44 leaves, max depth = 14, train score: 0.96395, val score: 0.88974, in 0.129s
[176/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96418, val score: 0.88954, in 0.129s
[177/1000] 1 tree, 44 leaves, max depth = 15, train score: 0.96436, val score: 0.88947, in 0.140s
[178/1000] 1 tree, 44 leaves, max depth = 18, train score: 0.96457, val score: 0.88965, in 0.198s
[179/1000] 1 tree, 4

(0.9586182585466435, 0.8625220675501433)